In [ ]:
import Pkg

Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("StatsBase")
Pkg.add("GLM")
Pkg.add("StatsModels")
Pkg.add("Combinatorics")


In [2]:
using CSV
using DataFrames
using StatsBase
using StatsModels
using Combinatorics
using GLM

using Printf

## Data Loading

In [3]:
file_path = joinpath(@__DIR__, "../dat/bottle.csv")
calcofi_data = CSV.read(file_path, DataFrame)

# Drop non numeric columns and rows with missing values
columns_to_drop = [:Sta_ID, :Depth_ID]
calcofi_data = select(calcofi_data, Not(columns_to_drop))

# Get the shape of the dataset
n_rows, n_cols = size(calcofi_data)
println("The dataset has $n_rows rows and $n_cols columns.")

The dataset has 864863 rows and 72 columns.


## Linear Regression

### Finding the optimal predictors

In [4]:
function data_missing_percentage(df::DataFrame)
    # Calculate the percentage of missing values for each column
    missing_values = map(col -> count(ismissing, col), eachcol(df))
    missing_percentages = missing_values ./ n_rows .* 100
    return missing_percentages
end

function delete_columns(df::DataFrame, threshold::Float64)
    # Get missing percentages
    missing_percentages = data_missing_percentage(df)

    # Identify columns to drop based on missing percentage threshold
    cols_to_drop = names(df)[missing_percentages .> threshold]

    # Drop the identified columns 
    cleaned_data = select(df, Not(cols_to_drop))
    return cleaned_data
end

function remove_outliers_IQR(df::DataFrame)
    outlier_indices = Set{Int}()  # Set to store indices of rows to remove

    #Calculate IQR and bounds for each numeric column
    for col in names(df)
        if eltype(df[!, col]) <: Real  # Check if column is numeric
            Q1 = quantile(skipmissing(df[!, col]), 0.25)
            Q3 = quantile(skipmissing(df[!, col]), 0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Identify outlier indices for this column
            for (i, value) in enumerate(df[!, col])
                if !ismissing(value) && (value < lower_bound || value > upper_bound)
                    push!(outlier_indices, i)
                end
            end
        end
    end

    # Remove rows with indices in the outlier set
    cleaned_df = df[setdiff(1:nrow(df), collect(outlier_indices)), :]
    return cleaned_df
end

function delete_columns_by_correlation(df::DataFrame, target_col::String, threshold::Float64)
    # Calculate correlations with the target column
    correlations = Dict{String, Float64}()
    for col in names(df)
        if col != target_col  # Skip the target column itself
            indices = findall(row -> !ismissing(row[target_col]) && !ismissing(row[col]), eachrow(df))
            if length(indices) > 1  # Ensure enough valid rows for correlation
                temp_df = df[indices, :]
                correlations[col] = cor(temp_df[:, target_col], temp_df[:,col])
            end
        end
    end

    # Filter columns based on correlation threshold
    high_corr_cols = filter(col -> abs(correlations[col]) ≥ threshold, keys(correlations))
    high_corr_cols = [target_col; high_corr_cols...]  # Include the target column in the result

    # Return the filtered DataFrame
    return select(df, high_corr_cols)

end;

In [5]:
missing_vals_threshold = 10.0

target_column = "T_degC"
tc_cor_threshold = 0.2

cleaned_data = delete_columns(calcofi_data, missing_vals_threshold)
cleaned_data = remove_outliers_IQR(cleaned_data)
reduced_df = delete_columns_by_correlation(cleaned_data, target_column, tc_cor_threshold)

predictors = names(reduced_df, Not(target_column))
n_predictors = length(predictors)
println("Final number of predictors: $n_predictors")

Final number of predictors: 11


In [6]:
function adjusted_r2(model, n, p)
    r = r2(model)  # R² from the model
    return 1 - ((1 - r) * (n - 1) / (n - p - 1))  # Adjusted R² formula
end;

In [43]:
target_column = "T_degC"
predictors = names(reduced_df, Not(target_column))

# Initialize storage for results
best_combination = []
best_adj_r2 = -Inf  # Start with the lowest possible adjusted R²

# Total number of rows in the DataFrame
n_rows = nrow(reduced_df)

# Test every combination of predictors
for k in 1:length(predictors)
    for combo in combinations(predictors, k)
        # Dynamically construct the formula
        formula = Term(Symbol(target_column)) ~ sum(Term(Symbol(p)) for p in combo)
        
        # Fit the model
        model = lm(formula, reduced_df)
        
        # Calculate adjusted R²
        adj_r2 = adjusted_r2(model, n_rows, length(combo))
        
        # Update the best model if current adjusted R² is higher
        if adj_r2 > best_adj_r2
            best_adj_r2 = adj_r2
            best_combination = combo
        end
    end
end

println("Best combination of predictors: ", best_combination)
println("Best adjusted R²: ", best_adj_r2)


Best combination of predictors: ["R_TEMP", "R_Depth", "Depthm", "R_PRES"]
Best adjusted R²: 0.999999977516001


Best combination of predictors: R_TEMP, R_Depth, Depthm, R_PRES

Best adjusted $R²$ : $0.999999977516001$